In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from datetime import datetime, timedelta

In [2]:
from langchain_google_community import CalendarToolkit
from langchain_google_community.calendar.utils import (
    build_resource_service,
    get_google_credentials,
    )

In [3]:
credentials = get_google_credentials(
    token_file="../backend/config/token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="../backend/config/client_secret_39562377782-nge5sdugil9eurkbgn54temjtgq06tbh.apps.googleusercontent.com.json",
)
credentials

In [4]:
api_resource = build_resource_service(credentials=credentials)
api_resource

In [22]:
def event_dict(calendar_id: str, date: datetime):
    """특정 날짜의 이벤트를 딕셔너리 형태로 반환"""
    time_min = date.isoformat() + "Z"
    time_max = (date + timedelta(days=1)).isoformat() + "Z"

    events_result = api_resource.events().list(
        calendarId=calendar_id,
        timeMin=time_min,
        timeMax=time_max,
        singleEvents=True,
        orderBy="startTime"
    ).execute()
    
    events = events_result.get("items", [])
    print(events)
    results  = dict()
    for event in events:
        results[event.get('summary')] = event.get('id')

    return results
calendar_id = "jongbaekim0710@gmail.com"
date = datetime(year=2025, month=10, day=28)
res_event_dict = event_dict(calendar_id=calendar_id, date=date)
res_event_dict


[{'kind': 'calendar#event', 'etag': '"3509740377148158"', 'id': '2p5a7d5h718i3ou6e3uf0bbhqe', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=MnA1YTdkNWg3MThpM291NmUzdWYwYmJocWUgam9uZ2JhZWtpbTA3MTBAbQ', 'created': '2025-06-12T22:21:49.000Z', 'updated': '2025-08-10T23:56:28.574Z', 'summary': '시계열교육(성덕,동현,수연,세은)', 'colorId': '4', 'creator': {'email': 'jongbaekim0710@gmail.com', 'self': True}, 'organizer': {'email': 'jongbaekim0710@gmail.com', 'self': True}, 'start': {'date': '2025-10-27'}, 'end': {'date': '2025-10-29'}, 'transparency': 'transparent', 'iCalUID': '2p5a7d5h718i3ou6e3uf0bbhqe@google.com', 'sequence': 0, 'reminders': {'useDefault': False, 'overrides': [{'method': 'popup', 'minutes': 30}, {'method': 'email', 'minutes': 1890}]}, 'eventType': 'default'}, {'kind': 'calendar#event', 'etag': '"3521337350435998"', 'id': '606820ikupqcstfb1u1pu9m211', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=NjA2ODIwaWt1cHFjc3RmYjF1MXB

{'시계열교육(성덕,동현,수연,세은)': '2p5a7d5h718i3ou6e3uf0bbhqe',
 '저녁약속(유라/태경)': '606820ikupqcstfb1u1pu9m211',
 '재택(신정수)': 'fl3hkg5g0c1kon4vi2dav5s78c',
 '삼호정기회의': '3sdq9as4jmga4kac385um8n42l_20251028T000000Z',
 '재택(성덕)': '3668tf3tk90ghls876qf5a8t9t',
 '하계휴가(정수)': '68ojiopg69h6cb9o68oj8b9k6cq66bb170rj8b9o6sp64e1j6ti38opnc8',
 '연차(최정)': 'ds76rhgkb9ldafpngtvt695nt8',
 '거점(강정훈, 동현)': 'fsaen6asi28igtl2mv14a7pmag',
 'AIC미팅': 'rcbnm6i0thj4qbv78eoq9a0of0'}

In [37]:
def add_meeting_event(calendar_id: str, start_datetime: datetime, summary: str = "회의", all_day: bool = None):
    """일정 추가 함수"""
    # all_day 값이 주어지지 않으면 start_datetime의 시간 여부로 자동 판정
    if all_day is None:
        all_day = (start_datetime.time() == datetime.min.time())  # 00:00:00이면 하루 종일로 간주

    if all_day:
        event = {
            "summary": summary,
            "start": {
                "date": start_datetime.date().isoformat(),
                "timeZone": "Asia/Seoul",  # ✅ 타임존 추가
            },
            "end": {
                "date": (start_datetime.date() + timedelta(days=1)).isoformat(),
                "timeZone": "Asia/Seoul",  # ✅ 타임존 추가
            },
        }
    else:
        end_datetime = start_datetime + timedelta(hours=1)
        event = {
            "summary": summary,
            "start": {
                "dateTime": start_datetime.isoformat(),
                "timeZone": "Asia/Seoul",
            },
            "end": {
                "dateTime": end_datetime.isoformat(),
                "timeZone": "Asia/Seoul",
            },
        }
    created_event = api_resource.events().insert(calendarId=calendar_id, body=event).execute()
    print(f"일정이 추가되었습니다: {created_event.get('htmlLink')}")

add_meeting_event(calendar_id=calendar_id, start_datetime=datetime(2025, 10, 28), summary="연차", all_day=True)

일정이 추가되었습니다: https://www.google.com/calendar/event?eid=Yzc4MWRkMjg3a2RjcmpjcGFrMXRnN20zODAgam9uZ2JhZWtpbTA3MTBAbQ


In [39]:
def update_event(calendar_id: str, event_id: str, start_datetime: datetime, summary: str = "회의", all_day: bool = None):
    """이벤트 업데이트 함수"""

    try:
        # 기존 이벤트 가져오기
        event = api_resource.events().get(calendarId=calendar_id, eventId=event_id).execute()

        # all_day 값 자동 판정 (입력 안하면 기존 설정 유지)
        if all_day is None:
            all_day = (start_datetime.time() == datetime.min.time())

        # 기존 event 정보 유지 + 변경 적용
        event["summary"] = summary

        if all_day:
            event["start"] = {
                "date": start_datetime.date().isoformat(),
                "timeZone": "Asia/Seoul",
            }
            event["end"] = {
                "date": (start_datetime.date() + timedelta(days=1)).isoformat(),
                "timeZone": "Asia/Seoul",
            }
        else:
            end_datetime = start_datetime + timedelta(hours=1)
            event["start"] = {
                "dateTime": start_datetime.isoformat(),
                "timeZone": "Asia/Seoul",
            }
            event["end"] = {
                "dateTime": end_datetime.isoformat(),
                "timeZone": "Asia/Seoul",
            }

        updated_event = api_resource.events().update(
            calendarId=calendar_id,
            eventId=event_id,
            body=event
        ).execute()

        print(f"이벤트(ID: {event_id})가 업데이트되었습니다: {updated_event.get('htmlLink')}")

    except Exception as e:
        print(f"이벤트(ID: {event_id}) 수정 중 오류 발생: {e}")


calendar_id = "jongbaekim0710@gmail.com"
start_datetime =datetime(2025, 10, 29)
update_event(calendar_id=calendar_id, event_id="c781dd287kdcrjcpak1tg7m380", summary="회의", start_datetime=start_datetime)


이벤트(ID: c781dd287kdcrjcpak1tg7m380)가 업데이트되었습니다: https://www.google.com/calendar/event?eid=Yzc4MWRkMjg3a2RjcmpjcGFrMXRnN20zODAgam9uZ2JhZWtpbTA3MTBAbQ


In [40]:
def delete_event(calendar_id: str, event_id: str):
    """이벤트 삭제 함수"""
    try:
        api_resource.events().delete(calendarId=calendar_id, eventId=event_id).execute()
        print(f"이벤트(ID: {event_id})가 삭제되었습니다.")
    except Exception as e:
        print(f"이벤트 IF({event_id}) 삭제 중 오류 발생: {e}")

delete_event(calendar_id=calendar_id, event_id="c781dd287kdcrjcpak1tg7m380")

이벤트(ID: c781dd287kdcrjcpak1tg7m380)가 삭제되었습니다.


In [8]:
import os
import json
from datetime import datetime
from typing import List, Union
from typing import Optional
from groq import Groq
from pydantic import BaseModel, ValidationError, field_validator

In [50]:
def date_extracter(user_input:str) -> str:
    """사용자의 입력에서 날짜와 시간을 추출"""
    system_prompt = f"""
    당신의 역할은 사용자의 입력에서 날짜 또는 시간과 관련된 단어를 포착후 String 형식으로 추출하는 것입니다.
    오늘은 {datetime.today()} 입니다.
    시간이 특정되지 않은 경우는 00:00 처리해주세요.
    
    <User Input>
    {user_input}
    </User Input>

    <Example>
    - user input: 10월 30일 오후 2시의 주요 일정을 브리핑
    - output : :2025-10-30, 14:00
    </Example>

    사용자의 입력에 응답하지 말고,
    입력에 포함된 날짜와 시간만 반환하고, 다른 부사적인 정보는 제외해주세요.

    """

    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",  # JSON 출력에 더 적합한 모델
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            temperature=0.0,  # 약간의 창의성 허용
            max_tokens=1000
        )
        res = response.choices[0].message.content        
        # print(json_str)
    
        return res

    except Exception as e:
        print(f"Error processing request: {str(e)}")
        return None
    

res_event_list = date_extracter(user_input="오늘 일정을 회사의 비서가 브리핑 하듯이 친절하게 설명해주세요")
print(res_event_list)


2025-10-28, 00:00


In [51]:
date_str = "2025-10-28, 00:00"
dt = datetime.strptime(date_str, "%Y-%m-%d, %H:%M")
dt

datetime.datetime(2025, 10, 28, 0, 0)

In [82]:
def get_schedules(calendar_id: str, user_input:str):
    """사용자 질문으로부터 날짜를 추출후 해당 날짜의 상세 일정을 딕셔너리 형태로 반환"""

    date_str = date_extracter(user_input=user_input)
    date = datetime.strptime(date_str, "%Y-%m-%d, %H:%M")
    time_min = date.isoformat() + "Z"
    time_max = (date + timedelta(days=1)).isoformat() + "Z"

    events_result = api_resource.events().list(
        calendarId=calendar_id,
        timeMin=time_min,
        timeMax=time_max,
        singleEvents=True,
        orderBy="startTime"
    ).execute()
    
    events = events_result.get("items", [])
    return events

user_input="내일 일정 브리핑"
res_event_dict = get_schedules(calendar_id=calendar_id, user_input=user_input)
res_event_dict

[{'kind': 'calendar#event',
  'etag': '"3523270260640542"',
  'id': '3668tf3tk90ghls876qf5a8t9t',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=MzY2OHRmM3RrOTBnaGxzODc2cWY1YTh0OXQgam9uZ2JhZWtpbTA3MTBAbQ',
  'created': '2025-10-28T07:05:30.000Z',
  'updated': '2025-10-28T07:05:30.320Z',
  'summary': '재택(성덕)',
  'creator': {'email': 'jongbaekim0710@gmail.com', 'self': True},
  'organizer': {'email': 'jongbaekim0710@gmail.com', 'self': True},
  'start': {'date': '2025-10-29'},
  'end': {'date': '2025-10-30'},
  'transparency': 'transparent',
  'iCalUID': '3668tf3tk90ghls876qf5a8t9t@google.com',
  'sequence': 0,
  'reminders': {'useDefault': False,
   'overrides': [{'method': 'popup', 'minutes': 30},
    {'method': 'email', 'minutes': 1890}]},
  'eventType': 'default'},
 {'kind': 'calendar#event',
  'etag': '"3522577260748350"',
  'id': '68ojiopg69h6cb9o68oj8b9k6cq66bb170rj8b9o6sp64e1j6ti38opnc8',
  'status': 'confirmed',
  'htmlLink': 'https://www.googl

In [86]:
def schedule_briefing(user_input:str, event_dict: dict) -> str:
    """사용자의 일정 브리핑"""
    system_prompt = f"""
    당신은 스마트하고 상냥한 일정관리 비서입니다.
    아래 내용을 참고하여 일정과 시간을 분석후 브리핑해주세요.
    장단기 일정을 구분하여 브리핑 해주세요.

    <Ref Schedules>
    {event_dict}

    <User Input>
    {user_input}
    
    반드시 주어진 일정에 있는 내용만 브리핑하고 친절하고 정중한 ton & manner를 지켜주세요.
    """

    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",  # JSON 출력에 더 적합한 모델
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            temperature=0.0,  # 약간의 창의성 허용
            max_tokens=1000
        )
        res = response.choices[0].message.content            
        return res

    except Exception as e:
        print(f"Error processing request: {str(e)}")
        return None
    

briefing_res = schedule_briefing(user_input=user_input, event_dict=res_event_dict)
print(briefing_res)


Error processing request: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01hrm502bcfvcas4j2ppdwhsxf` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 97903, Requested 2477. Please try again in 5m28.32s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
None


In [56]:
# def get_event_id(user_input:str, event_dict: dict) -> str:
#     """사용자 입력과 이벤트 딕셔너리를 비교하여 일치하는 이벤트 ID를 반환"""
#     system_prompt = f"""
#     You are an expert id finder.
#     Blow Ref Events are consist of title and id (key, value) pair dictionary.
#     Find the event that is as same as the input value among Ref Events presented and return the same item of that events.
    
#     <Ref Events>
#     {event_dict}

#     <User Input>
#     {user_input}

#     Output MUST be valid JSON containing array of item
#     """

#     try:
#         response = client.chat.completions.create(
#             model="llama-3.3-70b-versatile",  # JSON 출력에 더 적합한 모델
#             messages=[
#                 {"role": "system", "content": system_prompt},
#                 {"role": "user", "content": user_input}
#             ],
#             response_format={"type": "json_object"},
#             temperature=0.0,  # 약간의 창의성 허용
#             max_tokens=100
#         )
#         json_str = response.choices[0].message.content        
#         # print(json_str)
#         try:
#             parsed = json.loads(json_str)
#         except json.JSONDecodeError as e:
#             raise ValueError(f"Invalid JSON response: {e}")

#         # 단일 객체인 경우 리스트로 변환
#         if isinstance(parsed, dict):
#             parsed = [parsed]

#         return parsed if parsed else None

#     except Exception as e:
#         print(f"Error processing request: {str(e)}")
#         return None
    

# res_event_list = get_event_id(user_input="10월 28일 삼호 정기회의", event_dict=res_event_dict)
# res_event_list


In [57]:
# def is_new_event(user_input: str, event_list: List[str]) -> bool:
#     """사용자 입력이 기존 이벤트 목록에 없는 새로운 이벤트인지 확인"""
#     current_date = datetime.now().strftime("%Y-%m-%d")
    
#     # 기존 이벤트 리스트를 프롬프트에 포함
#     event_list_text = "\n".join(f"- {event}" for event in event_list)
    
#     system_prompt = f"""
#     You are an expert calendar assistant. Today's date is {current_date} (Asia/Seoul timezone).
#     You will determine if the user's input contains a new event **not already present** in the existing schedule.

#     <Instructions>
#     Compare the user's input to the following list of existing schedule items.
#     If the user's input describes an event that already exists (even approximately), respond with:
#         {{ "is_new": false }}
#     If the user's input describes a completely new event (not in the list), respond with:
#         {{ "is_new": true }}
#     Return only the JSON object and nothing else.

#     <Existing Events>
#     {event_list_text}
#     """

#     try:
#         response = client.chat.completions.create(
#             model="llama-3.3-70b-versatile",
#             messages=[
#                 {"role": "system", "content": system_prompt},
#                 {"role": "user", "content": user_input}
#             ],
#             response_format={"type": "json_object"},
#             temperature=0.3,
#             max_tokens=100
#         )
#         json_str = response.choices[0].message.content
#         print(json_str)
#         try:
#             result = json.loads(json_str)
#             return result.get("is_new", True)  # 기본값은 True (새 이벤트)
#         except json.JSONDecodeError as e:
#             raise ValueError(f"Invalid JSON response: {e}")

#     except Exception as e:
#         print(f"Error during event check: {str(e)}")
#         return True  # 에러 발생 시 기본적으로 새 이벤트라고 간주
    
# res = is_new_event(user_input="10월 28일 삼호 정기회의", event_list=res_event_list)
# res